In [1]:
!pip install -q flwr

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.15.1 requires protobuf<4.24,>=3.19.6, but you have protobuf 4.25.3 which is incompatible.


In [3]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.1 MB/s eta 0:00:00
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl.metadata (2.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.8/778.8 kB 7.6 MB/s eta 0:00:00a 0:00:01
Using cached thop-0.1.1.post2209072238-py3-none-any.whl (15 kB)
Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)


In [1]:
import flwr as fl
from ultralytics import YOLO
import hydra
from omegaconf import DictConfig, OmegaConf

/Users/clemensbandrock/.pyenv/versions/3.11.6/lib/python3.11/site-packages/thop/profile.py:12: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(torch.__version__) < LooseVersion("1.0.0"):


In [2]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cfg: DictConfig):
        self.model = YOLO("yolov10n.pt")
        self.cfg = cfg

    def get_parameters(self, config):
        return self.model.get_parameters()

    def set_parameters(self, parameters):
        self.model.set_parameters(parameters)

    def fit(self, parameters):
        self.set_parameters(parameters)

        self.model.train(data=self.cfg.data_path, epochs=self.cfg.epochs, device= self.cfg.device)

        #return parameters, number of training examples, 
        return self.get_parameters(), 1, {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)

        loss, accuracy = self.model.evaluate()

        return loss, 1, {"accuracy": accuracy}

In [25]:
cfg = OmegaConf.load("configs/base.yaml")

../../data/roadsigns/data.yaml


In [27]:
fl.client.start_client(server_address="[::]:8080", client=FlowerClient(cfg).to_client())

AttributeError: Can't get attribute 'YOLOv10DetectionModel' on <module 'ultralytics.nn.tasks' from '/Users/clemensbandrock/.pyenv/versions/3.11.6/lib/python3.11/site-packages/ultralytics/nn/tasks.py'>

In [29]:
from ultralytics import YOLO

# Load a pretrained YOLOv10n model
model = YOLO("yolov10n.yaml")
# Perform object detection on an image
results = model("image.jpg")

# Display the results
results[0].show()

FileNotFoundError: 'yolov10n.yaml' does not exist